In [18]:
import torch
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import random
from torch import Tensor
from lightly.models import utils
from typing import Optional, List,Tuple, Dict
from transformers import ViTForImageClassification
from sklearn.metrics import roc_auc_score, average_precision_score
from pytorch_lightning.loggers import CSVLogger
import csv

In [489]:
def evaluate_new(df):
    yt =np.array([np.array(x) for x in df['y_truth'].values])
    yp =np.array([np.array(x) for x in df['y_pred'].values])
    auroc = roc_auc_score(yt, yp)
    auprc = average_precision_score(yt, yp)
    return auprc, auroc

def bootstraping_eval(df, num_iter):
    """This function samples from the testing dataset to generate a list of performance metrics using bootstraping method"""
    auroc_list = []
    auprc_list = []
    for _ in range(num_iter):
        sample = df.sample(frac=1, replace=True)
        auprc, auroc = evaluate_new(sample)
        auroc_list.append(auroc)
        auprc_list.append(auprc)
    return auprc_list, auroc_list

def computing_confidence_intervals(list_,true_value):

    """This function calcualts the 95% Confidence Intervals"""
    delta = (true_value - list_)
    list(np.sort(delta))
    delta_lower = np.percentile(delta, 97.5)
    delta_upper = np.percentile(delta, 2.5)

    upper = true_value - delta_upper
    lower = true_value - delta_lower
    return (upper,lower)

def get_model_performance(df):
    test_auprc, test_auroc = evaluate_new(df)
    auprc_list, auroc_list = bootstraping_eval(df, num_iter=10)
    upper_auprc, lower_auprc = computing_confidence_intervals(auprc_list, test_auprc)
    upper_auroc, lower_auroc = computing_confidence_intervals(auroc_list, test_auroc)
    print("\n--------------")
    text_a=str(f"AUROC {round(test_auroc, 3)} ({round(lower_auroc, 3)}, {round(upper_auroc, 3)}) CI 95%")
    text_b=str(f"AUPRC {round(test_auprc, 3)} ({round(lower_auprc, 3)}, {round(upper_auprc, 3)}) CI 95% ")
    print(text_a,'\n')
    print(text_b,'\n')
    summary = {'test_auroc':np.round(test_auroc,3),
               'lower_auroc':np.round(lower_auroc,3),
               'upper_auroc':np.round(upper_auroc,3),
               'test_auprc':np.round(test_auprc,3),
               'lower_auprc':np.round(lower_auprc,3),
               'upper_auprc':np.round(upper_auprc,3),
               'auroc_text':text_a,
               'auprc_text':text_b}
    
#     final = pd.DataFrame(summary,index=[0])
#     final.to_csv(os.path.join(summary_path,'summary.csv'),index=False)
    #return summary

In [335]:
class EvaluationModel(pl.LightningModule):
    def __init__(self,
                backbone:nn.Module,
                learning_rate: float =  1e-3,
                weight_decay: float = 0.0,
                output_dim: int = 14,
                freeze: int = 0,
                max_epochs: int = 50,
                mask_ratio: float = 0.15,
                scheduler: str = 'cosine',
                summary_path: int  = './models'
                ) -> None:
        super().__init__()

        # self.save_hyperparameters() 
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.max_epochs = max_epochs
        self.output_dim = output_dim
        self.mask_ratio = mask_ratio
        self.backbone = backbone
        self.scheduler = scheduler
        self.summary_path = summary_path
        
        self.train_step_preds = []
        self.train_step_label = []
        
        self.val_step_preds = []
        self.val_step_label = []
        
        self.test_step_preds = []
        self.test_step_label = []
        
     
        if freeze == 1:
            utils.deactivate_requires_grad(self.backbone)
        elif freeze == 0:
            utils.activate_requires_grad(self.backbone)
               
    def forward(self,
                x: Tensor
               ) -> Tensor:
        x = self.backbone(x)
        x = torch.sigmoid(x)
        return x
    
    

    def training_step(self, 
                      batch: List[Tensor], 
                      batch_idx: int
                     ) -> float:
        
        input, label = batch
        prediction = self.forward(input)
        
        self.train_step_label.append(label)
        self.train_step_preds.append(prediction)

        loss = nn.BCELoss()(prediction, label)            
        self.log("train_loss", loss, on_epoch= True,on_step=False , logger=True, prog_bar=True)
        
        return {'loss':loss,
                'pred':prediction,
                'label':label}

    def on_train_epoch_end(self) -> None:


        
        y = torch.cat(self.train_step_label).detach().cpu()
        pred = torch.cat(self.train_step_preds).detach().cpu()

        auroc = np.round(roc_auc_score(y, pred), 4)
        auprc = np.round(average_precision_score(y, pred), 4)   
        self.log('train_auroc',auroc, on_epoch=True, on_step=False,logger=True, prog_bar=True)
        self.log('train_auprc',auprc, on_epoch=True, on_step=False,logger=True, prog_bar=True)      
        self.train_step_label.clear()
        self.train_step_preds.clear()
        
    def validation_step (self, 
                      batch: List[Tensor], 
                      batch_idx: int
                     ) -> float:
        
        input,label = batch
        prediction = self.forward(input) 
        
        self.val_step_label.append(label)
        self.val_step_preds.append(prediction)

        loss = self._bce_loss(prediction, label,mode='val')       
        self.log("val_loss", loss, on_epoch= True,on_step=False,logger=True, prog_bar=True)

        return {'loss':loss,
                'pred':prediction,
                'label':label}

    def on_validation_epoch_end(self,*arg, **kwargs) -> None:
        
        y = torch.cat(self.val_step_label).detach().cpu()
        pred = torch.cat(self.val_step_preds).detach().cpu()

        auroc = np.round(roc_auc_score(y, pred), 4)
        auprc = np.round(average_precision_score(y, pred), 4)   
        self.log('val_auroc',auroc, on_epoch=True, on_step=False, logger=True, prog_bar=True)
        self.log('val_auprc',auprc, on_epoch=True, on_step=False, logger=True, prog_bar=True)    
        self.val_step_label.clear()
        self.val_step_preds.clear()
        
    def test_step(self, 
                  batch: List[Tensor], 
                  batch_idx: int
                 ) -> float:
        input, label = batch
        prediction = self.forward(input)
        
        self.test_step_label.append(label)
        self.test_step_preds.append(prediction)
        loss = self._bce_loss(prediction, label,mode='test')
        self.log("test_loss", loss, on_epoch= True,on_step=False , logger=True, prog_bar=True)

        
        return {'loss':loss,
                'pred':prediction,
                'label':label}

    def on_test_epoch_end(self,*arg, **kwargs) -> None:
        y = torch.cat(self.test_step_label).detach().cpu()
        pred = torch.cat(self.test_step_preds).detach().cpu()


        auroc = np.round(roc_auc_score(y, pred), 4)
        auprc = np.round(average_precision_score(y, pred), 4)   
        self.log('test_auroc',auroc, on_epoch=True, on_step=False, logger=True)
        self.log('test_auprc',auprc, on_epoch=True, on_step=False, logger=True) 
        
        df = pd.DataFrame()
        df['y_truth'] = y.tolist()
        df['y_pred'] = pred.tolist()
        get_model_performance(df)
        
        
        self.test_step_label.clear()
        self.test_step_preds.clear()

    def configure_optimizers(self):

        optimizer = optim.Adam(params=self.parameters(), 
                                   lr=self.learning_rate, 
                                   weight_decay=self.weight_decay
                                   )
        if self.scheduler == 'cosine':
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                         eta_min=0,
                                                         T_max=self.max_epochs
                                                         )
            return {'optimizer': optimizer,
                    'lr_scheduler': scheduler,
                   }
        
        elif self.scheduler == 'reduce':
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                         mode='max',
                                                         factor=.5,
                                                         patience=3,
                                                         )   
            return {'optimizer': optimizer,
                    'lr_scheduler': scheduler,
                    'monitor': "val_auroc"
                   }
    

    
    def _bce_loss(self, preds, y,mode='train'):
        loss = nn.BCELoss()(preds, y)
        if torch.is_tensor(y):
            y = y.detach().cpu().numpy()
        return loss

In [336]:
import os
import glob
import torch
import numpy as np
import pandas as pd 
import pytorch_lightning as pl
import torchvision.transforms as T

from PIL import Image
from typing import Optional
from torch.utils.data import Dataset, DataLoader

In [337]:
def preprocess(data_dir:str,
               paths: list,
               split:str,
               ) -> Tuple[List]:
    CLASSES  = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
                'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
                'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
                'Pneumonia', 'Pneumothorax', 'Support Devices']

    filenames_to_path = {path.split('/')[-1].split('.')[0]: path for path in paths}
    metadata = pd.read_csv(os.path.join(data_dir,'mimic-cxr-2.0.0-metadata.csv'))
    labels = pd.read_csv(os.path.join(data_dir,'mimic-cxr-2.0.0-chexpert.csv'))
    labels[CLASSES] = labels[CLASSES].fillna(0)
    labels = labels.replace(-1.0, 0.0)
    splits = pd.read_csv(os.path.join(data_dir,'mimic-cxr-ehr-split.csv'))
    metadata_with_labels = metadata.merge(labels[CLASSES+['study_id'] ], how='inner', on='study_id')
    filesnames_to_labels = dict(zip(metadata_with_labels['dicom_id'].values, metadata_with_labels[CLASSES].values))
    filenames_loaded = splits.loc[splits.split==split]['dicom_id'].values
    
    filenames_loaded = [filename  for filename in filenames_loaded if filename in filesnames_to_labels]

    return filenames_to_path, filenames_loaded, filesnames_to_labels

In [338]:
IMAGENET_STAT = {"mean":torch.tensor([0.4884, 0.4550, 0.4171]),
                 "std":torch.tensor([0.2596, 0.2530, 0.2556])}
train_transforms = T.Compose([T.Resize(256),
                              T.RandomHorizontalFlip(),
                              T.RandomAffine(degrees=45, scale=(.85, 1.15), shear=0, translate=(0.15, 0.15)),
                              T.CenterCrop(224),
                              T.ToTensor(),
                              T.Normalize(mean=IMAGENET_STAT["mean"],
                                          std=IMAGENET_STAT["std"])                                                                   
                            ])


val_test_transforms = T.Compose([T.Resize(256),
                                 T.CenterCrop(224),
                                 T.ToTensor(),
                                 T.Normalize(mean=IMAGENET_STAT["mean"],
                                             std=IMAGENET_STAT["std"])                                                                
                            ])

In [339]:
class MIMICCXR(Dataset):
    def __init__(self, 
                 paths: str,
                 data_dir: str, 
                 transform: Optional[T.Compose] = None, 
                 split: str = 'validate',
                 percentage:float = 1.0
                 ) -> None:
        self.data_dir = data_dir
        self.transform = transform
        self.filenames_to_path, \
        self.filenames_loaded, \
        self.filesnames_to_labels = preprocess(data_dir=self.data_dir,
                                               paths=paths,
                                               split=split
                                              )
        limit = (round(len(self.filenames_loaded) * percentage))
        self.filenames_loaded = random.sample(self.filenames_loaded,limit)
 
        
    def __getitem__(self, index):
        if isinstance(index, str):
            img = Image.open(self.filenames_to_path[index]).convert('RGB')
            labels = torch.tensor(self.filesnames_to_labels[index]).float()

            if self.transform is not None:
                img = self.transform(img)
            return img, labels
        
        filename = self.filenames_loaded[index]
        
        img = Image.open(self.filenames_to_path[filename]).convert('RGB')

        labels = torch.tensor(self.filesnames_to_labels[filename]).float()
        
            
        if self.transform is not None:
            img = self.transform(img)
        return img, labels

    def __len__(self):
        return len(self.filenames_loaded)

In [340]:
class CheXpert(Dataset):
    def __init__(self, 
                 data_path: str,
                 transform: Optional[T.Compose] = None, 
                 split: str = 'test',
                 ) -> None:
        
        self.transform = transform
        
        self.data = pd.read_csv(data_path)

    
        
    def __getitem__(self, index):
        img = Image.open(self.data['Path'][index]).convert('RGB')
        labels = self.data.iloc[index,1:].to_numpy().astype('float32')
        img = self.transform(img)
        return img, labels

    def __len__(self):
        return len(self.data)

In [341]:
# chexpert = CheXpert(data_path='./chexert_test.csv',
#                     transform=val_test_transforms,
#                     split='test')

In [342]:
# chexpert_dataloader = DataLoader(dataset=chexpert,
#                              batch_size=688,
#                              shuffle=False,
#                              num_workers=24,
#                              pin_memory=True,
#                              drop_last=False
#                              )

In [343]:
# next(iter(chexpert_dataloader))

In [344]:
# data_dir = '/scratch/fs999/shamoutlab/data/physionet.org/files/mimic-cxr-jpg/2.0.0'

In [345]:
# paths = glob.glob(os.path.join(data_dir,'resized','**','*.jpg'), recursive=True)
# train_dataset = MIMICCXR(paths=paths,
#                          data_dir= data_dir, 
#                          split='train', 
#                          transform=train_transforms,
#                          percentage=0.05
#                          )
# val_dataset = MIMICCXR(paths=paths,
#                         data_dir=data_dir, 
#                         split='validate', 
#                         transform=val_test_transforms,
#                         percentage=0.05
#                         )
# test_dataset = MIMICCXR(paths=paths,
#                         data_dir=data_dir, 
#                         split='test', 
#                         transform=val_test_transforms,
#                         )

In [346]:
# train_dataloader = DataLoader(dataset=train_dataset,
#                              batch_size=64,
#                              shuffle=True,
#                              num_workers=24,
#                              pin_memory=True,
#                              drop_last=True
#                              )

# val_dataloader = DataLoader(dataset=val_dataset,
#                              batch_size=64,
#                              shuffle=False,
#                              num_workers=24,
#                              pin_memory=True,
#                              drop_last=True
#                              )

# test_dataloader = DataLoader(dataset=test_dataset,
#                              batch_size=64,
#                              shuffle=False,
#                              num_workers=24,
#                              pin_memory=True,
#                              drop_last=True
#                              )

In [370]:
chexpert = CheXpert(data_path='./chexert_test2.csv',
                    transform=val_test_transforms)

In [371]:
chexpert_dataloader = DataLoader(dataset=chexpert,
                             batch_size=1,
                             shuffle=False,
                             num_workers=24,
                             pin_memory=True,
                             drop_last=False
                             )

In [372]:
from torchvision.models.vision_transformer import VisionTransformer

In [449]:
backbone = VisionTransformer(image_size=224,
                             patch_size=16,
                             num_layers=12,
                             num_heads=6,
                             hidden_dim=192,
                             mlp_dim=192*4)
# backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
# backbone = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')

In [450]:
# checkpoint_dir = '/scratch/sas10092/ChexMSN/models/lightning_logs/version_33/checkpoints/epoch=99-step=587900.ckpt'
# all_weights = torch.load(checkpoint_dir,map_location='cpu')['state_dict']


In [451]:
# def parse_weights(weights: Dict[str,torch.Tensor]) -> Dict[str,torch.Tensor]:
    
#     for k in list(weights.keys()):

#         if k.startswith('backbone.'):
            
#             if k.startswith('backbone.') and not k.startswith('backbone.heads'):
                
#                 weights[k[len("backbone."):]] = weights[k]
                
#         del weights[k]
# #     del weights['class_token'] 
# #     del weights['encoder.pos_embedding']    
#     return weights

In [452]:
# all_weights

In [453]:
# weight = parse_weights(all_weights)

In [454]:
# weight['class_token'] = weight['class_token'][:,0:1]

In [455]:
# weight['encoder.pos_embedding'] = weight['encoder.pos_embedding'][:,1:]

In [456]:
# weight

In [457]:
# backbone.load_state_dict(weight,strict=False)

In [458]:
model = EvaluationModel(backbone=backbone,
                        learning_rate=0.0001,freeze=False)
# model = EvaluationModel.load_from_checkpoint(checkpoint_dir)

In [459]:
model.backbone.heads.head = nn.Linear(in_features=192,
                                      out_features=model.output_dim)
model.backbone.heads.head

Linear(in_features=192, out_features=14, bias=True)

In [460]:
# logger = CSVLogger(save_dir='/scratch/sas10092/ChexMSN/notebooks/low/',version=os.getenv('SLURM_JOB_ID'))

In [461]:
trainer = pl.Trainer(max_epochs=1,
                     num_sanity_val_steps=0)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [511]:
# trainer.fit(model=model, train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)
trainer.test(model=model,
             dataloaders=chexpert_dataloader,
            ckpt_path='/scratch/sas10092/ChexMSN/baselines/all-exp/logs/age-norm-only/tiny/lightning_logs/version_9/checkpoints/epoch=11-step=60972.ckpt')

Restoring states from the checkpoint path at /scratch/sas10092/ChexMSN/baselines/all-exp/logs/age-norm-only/tiny/lightning_logs/version_9/checkpoints/epoch=11-step=60972.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /scratch/sas10092/ChexMSN/baselines/all-exp/logs/age-norm-only/tiny/lightning_logs/version_9/checkpoints/epoch=11-step=60972.ckpt


Testing DataLoader 0: 100%|██████████| 612/612 [00:05<00:00, 108.38it/s]
--------------
AUROC 0.846 (0.836, 0.866) CI 95% 

AUPRC 0.492 (0.469, 0.531) CI 95%  

Testing DataLoader 0: 100%|██████████| 612/612 [00:05<00:00, 102.59it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auprc                 0.4921
       test_auroc                 0.8461
        test_loss           0.38194531202316284
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.38194531202316284,
  'test_auroc': 0.8461,
  'test_auprc': 0.4921}]

In [502]:
data = pd.read_csv('chexert_test.csv')

In [334]:
data.columns

Index(['Path', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
       'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity',
       'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia',
       'Pneumothorax', 'Support Devices'],
      dtype='object')

In [77]:
data['Sum'] = data.sum(axis=1)

In [82]:
data[data.Sum ==0.0]

,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,Sum
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
yt = np.array([1,1,1])
yp = [1,0,0]

In [179]:
yt.sum(axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1